In [ ]:
import pandas as pd
import os

labels_train=pd.read_csv("labels_train.csv")
labels_val=pd.read_csv("labels_val.csv")
print(labels_train.head())
print(labels_val.head())
# Print shape
print(labels_train.shape)
print(labels_val.shape)

In [ ]:

# For latitude and longitude columns, print the distribution
print("Latitude Distribution:")
print(labels_train['latitude'].describe())
print("\nLongitude Distribution:")
print(labels_train['longitude'].describe())

# Boxplot for latitude and longitude
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.boxplot(x=labels_train['latitude'])
plt.title('Latitude Distribution')
plt.show()

plt.figure(figsize=(12, 6))
sns.boxplot(x=labels_train['longitude'])
plt.title('Longitude Distribution')
plt.show()

In [ ]:
def iqr_filter(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Apply IQR filtering on labels_train
labels_train = iqr_filter(labels_train, 'latitude')
labels_train = iqr_filter(labels_train, 'longitude')

# Apply IQR filtering on labels_val
labels_val = iqr_filter(labels_val, 'latitude')
labels_val = iqr_filter(labels_val, 'longitude')
# Now describe latitude and longitude again
plt.figure(figsize=(12, 6))
sns.boxplot(x=labels_train['latitude'])
plt.title('Latitude Distribution')
plt.show()

plt.figure(figsize=(12, 6))
sns.boxplot(x=labels_train['longitude'])
plt.title('Longitude Distribution')
plt.show()



In [ ]:
#Standardize latitude and longitude
from sklearn.preprocessing import RobustScaler
latlong_scaler = RobustScaler()
latlong_train = labels_train[['latitude', 'longitude']]
latlong_val = labels_val[['latitude', 'longitude']]

labels_train[['latitude', 'longitude']] = latlong_scaler.fit_transform(latlong_train)
labels_val[['latitude', 'longitude']] = latlong_scaler.transform(latlong_val)


In [ ]:
# Data is cleaned, now let's check the number of rows
print(labels_train.shape)
print(labels_val.shape)

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import convnext_tiny, ConvNeXt_Tiny_Weights
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import pandas as pd

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset
class SMAIDataset(Dataset):
    def __init__(self, preprocess_df, root_dir, transform=None):
        self.preprocess_df = preprocess_df
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.preprocess_df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.preprocess_df.iloc[idx, 0])
        image = Image.open(img_name).convert("RGB")
        latitude = self.preprocess_df.iloc[idx, 2]
        longitude = self.preprocess_df.iloc[idx, 3]
        if self.transform:
            image = self.transform(image)
        labels = torch.tensor([latitude, longitude], dtype=torch.float32)
        return image, labels

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=10):
    best_val_loss = float('inf')
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        train_loss = running_loss / len(train_loader.dataset)

        # Validation
        model.eval()
        val_running_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item() * images.size(0)

        val_loss = val_running_loss / len(val_loader.dataset)
        scheduler.step(val_loss)

        print(f"Epoch [{epoch+1}/{epochs}] - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, LR: {optimizer.param_groups[0]['lr']}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "best_latlong_model.pth")
            print(f"Best model saved at epoch {epoch+1}")
        if optimizer.param_groups[0]['lr'] < 5e-6:
            break

    return model


# Latitude-Longitude Prediction

In [ ]:

# Load pretrained weights & transform
weights = ConvNeXt_Tiny_Weights.DEFAULT
# transform = weights.transforms()
transform=transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
# Load model
model = convnext_tiny(weights=weights,stochastic_depth_prob=0.5)
num_features = model.classifier[2].in_features
model.classifier=nn.Sequential(
     nn.Flatten(),
     nn.LayerNorm(num_features, eps=1e-6),
     nn.Dropout(p=0.5),
     nn.Linear(num_features,2)
    )
model = model.to(device)

# Datasets
train_dataset = SMAIDataset(labels_train, 'images_train', transform=transform)
val_dataset = SMAIDataset(labels_val, 'images_val', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Loss, Optimizer, Scheduler
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=1, factor=0.5)

# Training
print("Training for latitude and longitude")
model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=40)

In [ ]:
model.eval()
all_outputs = []
total_loss = 0.0

with torch.no_grad():
    for i, (images,labels) in enumerate(val_loader):
        latitudes = labels[:, 0]
        longitudes = labels[:, 1]
        batch_start = i * val_loader.batch_size
        images = images.to(device)

        targets = torch.stack((latitudes, longitudes), dim=1)
        outputs = model(images)

        # Inverse transform
        outputs_np = outputs.cpu().numpy()
        targets_np = targets.numpy()

        outputs_unscaled = latlong_scaler.inverse_transform(outputs_np)
        targets_unscaled = latlong_scaler.inverse_transform(targets_np)

        # Back to tensor
        outputs_unscaled_tensor = torch.tensor(outputs_unscaled, dtype=torch.float32)
        targets_unscaled_tensor = torch.tensor(targets_unscaled, dtype=torch.float32)

        # Loss
        loss = criterion(outputs_unscaled_tensor, targets_unscaled_tensor)
        total_loss += loss.item() * images.size(0)

        # Add predictions with index as ID
        for j, (lat, lon) in enumerate(outputs_unscaled):
            all_outputs.append([batch_start + j, lat, lon])

avg_loss = total_loss / len(val_loader.dataset)
print(f"Validation MSE: {avg_loss:.4f}")

# Save
df = pd.DataFrame(all_outputs, columns=["id", "Latitude", "Longitude"])
df.to_csv("solution.csv", index=False)
